In [30]:
import PyPDF2 
import textract
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

filename = r'D7A5BDB1004EF70752B47DDA723E8DA1_labels.pdf' #Romaneio do Mercado Livre
pdfFileObj = open(filename,'rb') #open allows you to read the file.

#The pdfReader variable is a readable object that will be parsed (analisado).
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)

#Discerning the number of pages will allow us to parse through all the pages.
num_pages = pdfReader.numPages
count = 0
text = ""

#The while loop will read each page.
while count < num_pages:
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
    
#This if statement exists to check if the above library returned words. It's done because PyPDF2 cannot read scanned files.
if text != "":
    text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text.
else:
    text = textract.process(filename, method='tesseract', language='eng')

#Now we have a text variable that contains all the text derived from our PDF file. Type print(text) to see what it contains. It likely contains a lot of spaces, possibly junk such as '\n,' etc.

In [31]:
# Converter o texto em palavras chave

#The word_tokenize() function will break our text phrases into individual words.
tokens = word_tokenize(text)

#We'll create a new list that contains punctuation we wish to clean.
punctuations = ['(',')',';',':','[',']',',','#','-']

#We initialize the stopwords variable, which is a list of words like "The," "I," "and," etc. that don't hold much value as keywords.
stop_words = stopwords.words('english')

filtro_1 = ['Terçariol', 'Embalagens', 'LTDA']

#We create a list comprehension that only returns a list of words that are NOT IN stop_words and NOT IN punctuations.
keywords = [word for word in tokens if not word in stop_words and not word in punctuations and not word in filtro_1]

In [32]:
# Localizando as listas

i = 0
indice = []

for palavra in keywords:
    if palavra == 'Produtos':
        indice.append(i)
    i += 1

In [33]:
from itertools import islice

# Pegar as palavras das listas:
useful_words = keywords[indice[0]+1: len(keywords)] # lista de palavras contidas no romaneio

#Separar volume por volume

j = 0
indice_1 = []

for palavra in useful_words: #Separar o Volume pelo termo MEL0....
    if 'Venda' in palavra:
        indice_1.append(j)
    j += 1
    
numero_volumes = len(indice_1) # Número de volumes

volumes = list()
tamanhos = [] #Quantidade de palavras para cada volume

for k in range(numero_volumes): #Pegando os volumes
    if k < numero_volumes - 1:
        tamanhos.append(indice_1[k+1] - indice_1[k])
    elif k == numero_volumes - 1:
        tamanhos.append(len(useful_words) - indice_1[k]+1)

def getSublists(lst,lens):
    iter_lst = iter(lst)
    return [list(islice(iter_lst, x)) for x in lens]
volumes = (getSublists(useful_words,tamanhos)) #Lista de pedidos

In [34]:
#PADRÃO PRODUTOS
acoplado = 'Acoplado'
acoplado_kraft = 'kraft acoplado'
saco_kraft_M = 'Saco delivery'
saco_kraft_P = 'Saco P delivery'
saco_branco_M = 'Saco branco delivery'
saco_branco_P = 'Saco P branco delivery'
saquinho_batata_kraft = 'Saquinho de batata kraft'
saquinho_batata_barreira = 'Saquinho de batata barreira'
saquinho_talher_kraft = 'Talher saquinho kraft'
saquinho_talher = 'Taler Saquinho'
cx_pizza = 'CXS pizza'
cx_papelao_PP = 'CXS papelão pp'
cx_papelao_P = 'CXS papelão p'
cx_papelao_M = 'CSX papelão m'
glassine = 'Glassine'
guardanapo = 'Guardanapo sachê'
toalha = 'Toalha'
toalha_kraft = 'kraft toalha'
amostras = 'Amostras'

escrita_padronizada = []
quantidade_caixas = [] #Multier de quantas unidades do anúncio o cliente comprou
indice_qtde = []

for i in range(numero_volumes):#percorrendo número de volumes
    indice_qtde.append(volumes[i].index('Quantidade')) #Localizar onde vem a quantidade

for i in range(numero_volumes):# Pegando as quantidades
    quantidade_caixas.append(volumes[i][indice_qtde[i]+1])
    

#PERCORRER TODOS OS VOLUMES PEGANDO A ESCRITA
for i in range(numero_volumes):
    if 'Kit' not in volumes[i]:
        if 'Tam' in volumes[i]: #Checar SACO DELIVERY TAMANHO P
            if 'Pequeno' in volumes[i]: 
                if 'Kraft' in volumes[i]:# SACO DELIVERY KRAFT TAMANHO P
                    escrita_padronizada.append(i)
                    #escrita_padronizada.append(quantidade_caixas[i])
                    escrita_padronizada.append(saco_kraft_P)
                if 'Branco' in volumes[i]:# SACO DELIVERY BRANCO TAMANHO P
                    escrita_padronizada.append(i)
                    escrita_padronizada.append(saco_branco_P)
        if 'Tamanho' in volumes[i]: #Checar SACO DELIVERY TAMANHO P
            if 'Pequeno' in volumes[i]: 
                if 'Kraft' in volumes[i]:# SACO DELIVERY KRAFT TAMANHO P
                    escrita_padronizada.append(i)
                    #escrita_padronizada.append(quantidade_caixas[i])
                    escrita_padronizada.append(saco_kraft_P)
                if 'Branco' in volumes[i]:# SACO DELIVERY BRANCO TAMANHO P
                    escrita_padronizada.append(i)
                    escrita_padronizada.append(saco_branco_P)
        if 'Saquinho' not in volumes[i]:
            if 'Saco' in volumes[i]:#Checar SACO DELIVERY
                if 'Pequeno' not in volumes[i]:
                    if 'Kraft' in volumes[i]:#SACO DELIVERY M KRAFT
                        escrita_padronizada.append(i)
                        escrita_padronizada.append(saco_kraft_M)
                    if 'Branco' in volumes[i]:#SACO DELIVERY M BRANCO
                        escrita_padronizada.append(i)
                        escrita_padronizada.append(saco_branco_M)
            if 'Sacola' in volumes[i]:
                if 'Pequeno' not in volumes[i]:
                    if 'Kraft' in volumes[i]:
                        escrita_padronizada.append(i)
                        escrita_padronizada.append(saco_kraft_M)
                    else:
                        escrita_padronizada.append(i)
                        escrita_padronizada.append(saco_branco_M)
        if 'Acoplado' in volumes[i]:#PAPEL ACOPLADO
            if 'Kraft' in volumes[i]:#ACOPLADO KRAFT
                escrita_padronizada.append(i)
                escrita_padronizada.append(acoplado_kraft)
            if 'Kraft' not in volumes[i]:#ACOPLADO MONO
                escrita_padronizada.append(i)
                escrita_padronizada.append(acoplado)
        if 'Pizza' in volumes[i]:#CAIXAS DE PIZZA
            escrita_padronizada.append(i)
            escrita_padronizada.append(cx_pizza)
        if 'Papelão' in volumes[i]:#CAIXAS DE PAPELÃO
            if 'Pizza' not in volumes[i]:
                if 'Pp' in volumes[i]:#CAIXAS PAPELÃO PP
                    escrita_padronizada.append(i)
                    escrita_padronizada.append(cx_papelao_PP)
            if 'P' in volumes[i]:
                escrita_padronizada.append(i)
                escrita_padronizada.append(cx_papelao_P)
        if 'Manteiga' in volumes[i]:#Papel Glassine
            if 'Papel' in volumes[i]:
                if 'Acoplado' not in volumes[i]:
                    if 'Saquinho' not in volumes[i]:
                        escrita_padronizada.append(i)
                        escrita_padronizada.append(glassine)
        if 'Guardanapo' in volumes[i]:#Guardanapo sachê
            escrita_padronizada.append(i)
            escrita_padronizada.append(guardanapo)
        if 'Batata' in volumes[i]:#CHECAR O SAQUINHO DE BATATA
            if 'Kraft' in volumes[i]:#Kraft
                escrita_padronizada.append(i)
                escrita_padronizada.append(saquinho_batata_kraft)
            else:#Saquinho barreira
                escrita_padronizada.append(i)
                escrita_padronizada.append(saquinho_batata_barreira)
        if 'Talher' in volumes[i]:#Checar saqunhi de talher
            if 'Kraft' in volumes[i]:#Kraft
                escrita_padronizada.append(i)
                escrita_padronizada.append(saquinho_talher_kraft)
            else:#Barreira
                escrita_padronizada.append(i)
                escrita_padronizada.append(saquinho_talher)
    else:
        escrita_padronizada.append(i)
        escrita_padronizada.append('Kit')

# Retirando Volumes com mais de uma compra
    #Retirando 2 compras
for i in range(len(escrita_padronizada)):
    if escrita_padronizada.count(i) == 2:
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)
    #Retirando 3 compras
for i in range(len(escrita_padronizada)):
    if escrita_padronizada.count(i) == 3:
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)
for i in range(len(escrita_padronizada)):
    if escrita_padronizada.count(i) == 4:
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)
        escrita_padronizada.pop(i*2+2)

print(len(escrita_padronizada))
print(escrita_padronizada)

100
[0, 'Acoplado', 1, 'Saquinho de batata kraft', 2, 'Guardanapo sachê', 3, 'Saco delivery', 4, 'Acoplado', 5, 'Saco delivery', 6, 'Saco P delivery', 7, 'Guardanapo sachê', 8, 'Acoplado', 9, 'Saquinho de batata barreira', 10, 'Saco delivery', 11, 'Acoplado', 12, 'Saco delivery', 13, 'Saco P delivery', 14, 'Acoplado', 15, 'Guardanapo sachê', 16, 'Saco delivery', 17, 'Acoplado', 18, 'CXS papelão pp', 19, 'Kit', 20, 'Guardanapo sachê', 21, 'Guardanapo sachê', 22, 'Saquinho de batata kraft', 23, 'Guardanapo sachê', 24, 'Saco delivery', 25, 'Acoplado', 26, 'Glassine', 27, 'Talher saquinho kraft', 28, 'Guardanapo sachê', 29, 'kraft acoplado', 30, 'Talher saquinho kraft', 31, 'Acoplado', 32, 'Guardanapo sachê', 33, 'kraft acoplado', 34, 'Saquinho de batata kraft', 35, 'Guardanapo sachê', 36, 'Saco P delivery', 37, 'Acoplado', 38, 'Saquinho de batata kraft', 39, 'Acoplado', 40, 'Guardanapo sachê', 41, 'kraft acoplado', 42, 'Saco delivery', 43, 'Guardanapo sachê', 44, 'Guardanapo sachê', 45, '

In [35]:
indice_qtde_2 = []#Indice que o Unds aparece na escrita
volumes_unds = []#Volumes que contem Unds no texto
quantidade_caixas_2 = []#Número que aparece no texto
indice_unds = []#Número do Volume que corresponde a lista de Unds
indice_cxs = []#Número do Volume que corresponde as caixas
volumes_caixas = []#Volumes que contem Caixas no texto
indice_saches = []#número do Volume que corresponde aos sachês
volumes_saches = []#Volumes que contem Sachês no texto
indice_qtde_3 = []#Indice que Caixas aparecem na escrita
indice_qtde_4 = []#Indice que os Sachês aparecem na escrita
quantidade_caixas_3 = []#Número que aparece no texto
quantidade_caixas_4 = []#Número que aparece no texto
quantidade_total = []#números que aparecem no texto juntados
indice_qtde_05 = []#1000unds
indice_1000unds = []#1000unds
volumes_1000unds = []#1000unds
quantidade_caixas_05 = []#1.000unds
indice_qtde_06 = []#und
indice_und = []#und
volumes_und = []#und
quantidade_caixas_06 = []#und
indice_qtde_07 = []#Kit
indice_kit = []#kit
volumes_kit = []#kit
quantidade_caixas_07 = []#kit


        
for i in range(numero_volumes):# Pegando as qtds
    if 'Unds' in volumes[i]:# UNDS
        indice_qtde_2.append(volumes[i].index('Unds')) #Localizar os números
        indice_unds.append(i)
        volumes_unds.append(volumes[i])
    if 'Caixas' in volumes[i]: #CAIXAS
        indice_qtde_3.append(volumes[i].index('Caixas'))
        indice_cxs.append(i)
        volumes_caixas.append(volumes[i])
    if 'Sachês.delivery' in volumes[i]: #Sachês
        indice_qtde_4.append(volumes[i].index('Sachês.delivery'))
        indice_saches.append(i)
        volumes_saches.append(volumes[i])
    if '1.000unds' in volumes[i]:# UNDS fora do padrão
        indice_qtde_05.append(volumes[i].index('1.000unds'))
        indice_1000unds.append(i)
        volumes_1000unds.append(volumes[i])
    if 'Und' in volumes[i]:# UND
        indice_qtde_06.append(volumes[i].index('Und')) #Localizar os números
        indice_und.append(i)
        volumes_und.append(volumes[i])
    if 'Kit' in volumes[i]:#KIT
        indice_qtde_07.append(volumes[i].index('Kit')) #Localizar os números
        indice_kit.append(i)
        volumes_kit.append(volumes[i])
        

for i in range(len(volumes_unds)):# Pegando os números que aparecem no texto
    quantidade_caixas_2.append(volumes_unds[i][indice_qtde_2[i]-1])

for i in range(len(volumes_caixas)):
    quantidade_caixas_3.append(volumes_caixas[i][indice_qtde_3[i]-1])
    
for i in range(len(volumes_saches)):
    quantidade_caixas_4.append(volumes_saches[i][indice_qtde_4[i]-1])
    
for i in range(len(volumes_1000unds)):
    quantidade_caixas_05.append(1000)
    
for i in range(len(volumes_und)):# Pegando os números que aparecem no texto
    quantidade_caixas_06.append(volumes_und[i][indice_qtde_06[i]-1])
    
for i in range(len(volumes_kit)):# Pegando os números que aparecem no texto
    quantidade_caixas_07.append(volumes_und[i][indice_qtde_07[i]-1])

a = 0
b = 0
c = 0
d = 0
e = 0
f = 0

#Juntando as quantidades
for i in range(numero_volumes):#Pegando as quantidades na escrita das unidades com os índices
    if '250unds' in volumes[i]:
        quantidade_total.append(i)
        quantidade_total.append('250')
    if i in indice_unds:
        quantidade_total.append(i)
        quantidade_total.append(volumes[i][indice_qtde_2[c]-1])
        c += 1

    if i in indice_cxs:#Pegando as quantidades na escrita das caixas com os índices
        quantidade_total.append(i)
        quantidade_total.append(volumes[i][indice_qtde_3[a]-1])
        a += 1
        
    if i in indice_saches:#Pegando as quantidades na escrita dos saches com os índices
        quantidade_total.append(i)
        quantidade_total.append(volumes[i][indice_qtde_4[b]-1])
        b += 1
        
    if i in indice_1000unds:
        quantidade_total.append(i)
        quantidade_total.append(1000)
        d += 1
        
    if i in indice_und:
        quantidade_total.append(i)
        quantidade_total.append(volumes[i][indice_qtde_06[e]-1])
        e += 1
        
    if i in indice_kit:
        quantidade_total.append(i)
        quantidade_total.append(404)
        f += 1
    
print(quantidade_total)
print(len(quantidade_total))
quantidade_total = [str(x) for x in quantidade_total]

[0, '2.000', 1, '2.000', 2, '1.000', 3, '500', 4, '1.000', 5, 1000, 6, '250', 7, '2.000', 8, '500', 9, '2.000', 10, '250', 11, '500', 12, '250', 13, '500', 14, '500', 15, '2.000', 16, '250', 17, '1.000', 18, '75', 19, 404, 20, '2.000', 21, '1.000', 22, '2.000', 23, '2.000', 24, 1000, 25, '500', 26, '500', 27, '2.000', 28, '2.000', 29, '500', 30, '2.000', 31, '1.000', 32, '2.000', 33, '500', 34, '2.000', 35, '2.000', 36, '250', 37, '1.000', 38, '2.000', 39, '500', 40, '2.000', 41, '500', 42, '250', 43, '2.000', 44, '2.000', 45, '250', 46, '250', 47, '500', 48, '2000', 49, '2.000']
100


In [19]:
for i in range(len(quantidade_total)):
    quantidade_total.append(quantidade_total[i].replace('.',''))

print(len(quantidade_total))
print(quantidade_total)

116
['0', '1000', '1', '500', '2', '2.000', '3', '500', '4', '2.000', '5', '250', '6', '250', '7', '1.000', '8', '250', '9', '500', '10', '250', '11', '1.000', '12', '1000', '13', '1000', '14', '1.000', '15', '1000', '16', '1.000', '17', '2.000', '18', '2.000', '19', '2.000', '20', '1000', '21', '2.000', '22', '50', '23', '1000', '24', '1000', '25', '500', '26', '1.000', '27', '2.000', '28', '1.000', '0', '1000', '1', '500', '2', '2000', '3', '500', '4', '2000', '5', '250', '6', '250', '7', '1000', '8', '250', '9', '500', '10', '250', '11', '1000', '12', '1000', '13', '1000', '14', '1000', '15', '1000', '16', '1000', '17', '2000', '18', '2000', '19', '2000', '20', '1000', '21', '2000', '22', '50', '23', '1000', '24', '1000', '25', '500', '26', '1000', '27', '2000', '28', '1000']


In [20]:
quantidade_total = [float(x) for x in quantidade_total]

In [21]:
#PEGAR AS QUANTIDADES TOTAL E IDENTIFICAR O NUMERO DE CAIXAS CORRESPONDENTE
n_acoplado = 500
n_saco = 250
n_guardanapo = 1000
n_caixas = 25
n_saquinho = 2000
quantidade_verdadeira = [] #QUANTIDADE FINAL DE CAIXAS QUE VÃO SAIR DE CADA VOLUME

for i in range(len(quantidade_total)):
    if quantidade_total[i] <= 20:
        quantidade_total[i] = quantidade_total[i]*1000
        
quantidade_caixas = [int(x) for x in quantidade_caixas]

d = 0
e = 1

#Somando a quantidade do titulo
for i in range(numero_volumes):
    if 'Kit' in volumes[i]:
        quantidade_verdadeira.append(d)
        quantidade_verdadeira.append(404)
    else:
        if 'Saquinho' not in volumes[i]:
            if 'Saco' in volumes[i]: #Saco Delivery
                quantidade_verdadeira.append(d)
                quantidade_verdadeira.append((quantidade_total[e]/n_saco)*quantidade_caixas[i])
            if 'Sacola' in volumes[i]:
                quantidade_verdadeira.append(d)
                quantidade_verdadeira.append((quantidade_total[e]/n_saco)*quantidade_caixas[i])
        if 'Acoplado' in volumes[i]: # Acoplado
            quantidade_verdadeira.append(d)
            quantidade_verdadeira.append((quantidade_total[e]/n_acoplado)*quantidade_caixas[i])
        if 'Sachês.delivery' in volumes[i]: #Sachês
            quantidade_verdadeira.append(d)
            quantidade_verdadeira.append((quantidade_total[e]/n_guardanapo)*quantidade_caixas[i])
        if 'Papelão' in volumes[i]: #Caixas de papelão
            quantidade_verdadeira.append(d)
            quantidade_verdadeira.append((quantidade_total[e]/n_caixas)*quantidade_caixas[i])
        if 'Acoplado' not in volumes[i]: #Glassine
            if 'Papel' and 'Para' and 'Hamburguer' in volumes[i]:
                quantidade_verdadeira.append(d)
                quantidade_verdadeira.append((quantidade_total[e]/n_acoplado)*quantidade_caixas[i])
            if 'Glassine' in volumes[i]:
                quantidade_verdadeira.append(d)
                quantidade_verdadeira.append((quantidade_total[e]/n_acoplado)*quantidade_caixas[i])
        if 'Saquinho' in volumes[i]: #Saquinho de batata
            quantidade_verdadeira.append(d)
            quantidade_verdadeira.append((quantidade_total[e]/n_saquinho)*quantidade_caixas[i])

    d += 1
    e += 2

    
escrita_padronizada = [str(x) for x in escrita_padronizada] #Passando para string
quantidade_verdadeira = [int(x) for x in quantidade_verdadeira] #Passando para string
quantidade_verdadeira = [str(x) for x in quantidade_verdadeira] #Passando para string

indices_quantidade = []
for i in range(int(len(quantidade_verdadeira)/2)):
    indices_quantidade.append(quantidade_verdadeira[i*2])

for i in range(len(indices_quantidade)):
    if indices_quantidade.count(str(i)) == 2:
        quantidade_verdadeira.pop(i*2+2)
        quantidade_verdadeira.pop(i*2+2)

juntos = [] # Quantidade de Caixas/fardos + produto (Falta modelo do produto apenas)

for i in range(len(escrita_padronizada)):
    if i % 2 != 0:
        juntos.append(quantidade_verdadeira[i] + ' ' + escrita_padronizada[i])

for i in range(len(juntos)):
    print(juntos[i] + str(i))
print(quantidade_verdadeira)
print(len(quantidade_verdadeira))

4 Saco delivery0
2 Glassine1
2 Guardanapo sachê2
6 kraft acoplado3
2 Guardanapo sachê4
1 Saco delivery5
1 Saco delivery6
3 Guardanapo sachê7
1 Saco delivery8
4 Saco P delivery9
1 Saco delivery10
1 Guardanapo sachê11
4 Saco delivery12
4 Saco delivery13
2 Acoplado14
4 Saco delivery15
6 Acoplado16
1 Saquinho de batata kraft17
2 Guardanapo sachê18
2 Guardanapo sachê19
4 Saco delivery20
2 Guardanapo sachê21
0 Saco delivery22
2 Acoplado23
4 Saco delivery24
1 Glassine25
6 Acoplado26
2 Guardanapo sachê27
2 Acoplado28
['0', '4', '1', '2', '2', '2', '3', '6', '4', '2', '5', '1', '6', '1', '7', '3', '8', '1', '9', '4', '10', '1', '11', '1', '12', '4', '13', '4', '14', '2', '15', '4', '16', '6', '17', '1', '18', '2', '19', '2', '20', '4', '21', '2', '22', '0', '23', '2', '24', '4', '25', '1', '26', '6', '27', '2', '28', '2']
58


In [22]:
print(quantidade_verdadeira)
print(len(quantidade_verdadeira))
print(' ')
print(escrita_padronizada)

['0', '4', '1', '2', '2', '2', '3', '6', '4', '2', '5', '1', '6', '1', '7', '3', '8', '1', '9', '4', '10', '1', '11', '1', '12', '4', '13', '4', '14', '2', '15', '4', '16', '6', '17', '1', '18', '2', '19', '2', '20', '4', '21', '2', '22', '0', '23', '2', '24', '4', '25', '1', '26', '6', '27', '2', '28', '2']
58
 
['0', 'Saco delivery', '1', 'Glassine', '2', 'Guardanapo sachê', '3', 'kraft acoplado', '4', 'Guardanapo sachê', '5', 'Saco delivery', '6', 'Saco delivery', '7', 'Guardanapo sachê', '8', 'Saco delivery', '9', 'Saco P delivery', '10', 'Saco delivery', '11', 'Guardanapo sachê', '12', 'Saco delivery', '13', 'Saco delivery', '14', 'Acoplado', '15', 'Saco delivery', '16', 'Acoplado', '17', 'Saquinho de batata kraft', '18', 'Guardanapo sachê', '19', 'Guardanapo sachê', '20', 'Saco delivery', '21', 'Guardanapo sachê', '22', 'Saco delivery', '23', 'Acoplado', '24', 'Saco delivery', '25', 'Glassine', '26', 'Acoplado', '27', 'Guardanapo sachê', '28', 'Acoplado']


In [23]:
#PEGAR O MODELO DO PRODUTO#
#chaves_modelos = ['Tipo', 'Tipo_' , 'Cor', 'desenho'] 
indice_qtde_5 = []#Indice que o Tipo aparece na escrita
indice_qtde_6 = []#indice que o Tipo_ aparece na escrita
indice_qtde_7 = []#indice que a Cor aparece na escrita
indice_qtde_8 = []#Indice que o desenho aparece na escrita
indice_qtde_9 = []#Indice que a cor aparece na escrita
indice_qtde_10 = []#Indice que a Estampa aparece na escrita
indice_qtde_11 = []#Indice que Kraft aparece na escrita
indice_tipo = []#Número do Volume que corresponde a lista de Tipo
indice_tipo_ = []#Número do Volume que corresponde a lista de Tipo_
indice_cor = []#Número do Volume que corresponde a Cor
indice_desenho = []#Número do Volume que corresponde o desenho
indice_cor1 = []#Número do Volume que corresponde a cor
indice_estampa = []#Número do Volume que corresponde a Estampa
indice_kraft = []#Número do Volume que corresponde a Kraft
volumes_tipo = []#Volumes que contem Tipo no texto
volumes_tipo_ = []#Volumes que contem Tipo_ no texto
volumes_cor = []#Volumes que contem Cor no texto
volumes_desenho = []#Volumes que contem desenho no texto
volumes_cor1 = []#Volumes que contem cor no texto
volumes_estampa = []#Volumes que contem estampa no texto
volumes_kraft = []#Volumes que contem kraft no texto
modelos_tipo = []#Modelos que aparecenm após o tipo
modelos_tipo_ = []#Modelos que aparecem após o tipo_
modelos_cor = []#Modelos que aparecem após a cor
modelos_desenho = []#Modelos que aparecem após desenho
modelos_cor1 = []#Modelos que aparecenm após cor
modelos_estampa = []#Modelos que aparecem estampa
modelos_kraft = []#Modelos que aparecem kraftestampa

for i in range(numero_volumes):    
    if 'Tipo' in volumes[i]:
        indice_qtde_5.append(volumes[i].index('Tipo')) #indice que o Tipo aparece
        indice_tipo.append(i)
        volumes_tipo.append(volumes[i])
    
    if 'Tipo_' in volumes[i]:
        indice_qtde_6.append(volumes[i].index('Tipo_')) #indice que o Tipo_ aparece
        indice_tipo_.append(i)
        volumes_tipo_.append(volumes[i])
        
    if 'Cor' in volumes[i]:
        if 'Tipo_' not in volumes[i]:
            indice_qtde_7.append(volumes[i].index('Cor')) #indice que a Cor aparece
            indice_cor.append(i)
            volumes_cor.append(volumes[i])
        
    if 'desenho' in volumes[i]:
        if 'Tipo_' not in volumes[i]:
            indice_qtde_8.append(volumes[i].index('desenho')) #indice que o desenho aparece
            indice_desenho.append(i)
            volumes_desenho.append(volumes[i])
            
    if 'cor' in volumes[i]:
        if 'Tipo_' not in volumes[i]:
            indice_qtde_9.append(volumes[i].index('cor')) #indice que a Cor aparece
            indice_cor1.append(i)
            volumes_cor1.append(volumes[i])
    
    if volumes[i][indice_qtde[i]+2] == 'Estampa':
        indice_qtde_10.append(indice_qtde[i]+2) #indice que a estampa aparece
        indice_estampa.append(i)
        volumes_estampa.append(volumes[i])
        
    if volumes[i][indice_qtde[i]+2] == 'Kraft':
        indice_qtde_11.append(indice_qtde[i]+2) #indice que Kraft aparece
        indice_kraft.append(i)
        volumes_kraft.append(volumes[i])
    
#Identificando os modelos com seus respectivos indices

for i in range(len(volumes_tipo)):# Modelos que aparecem após o Tipo
    if volumes_tipo[i][indice_qtde_5[i]+2] == '6':
        modelos_tipo.append(indice_tipo[i])
        modelos_tipo.append('Sem estampa')
    if volumes_tipo[i][indice_qtde_5[i]+3] == 'CBJ':
        modelos_tipo.append(indice_tipo[i])
        modelos_tipo.append('CBJ')
    if volumes_tipo[i][indice_qtde_5[i]+3] == 'Saco':
        modelos_tipo.append(indice_tipo[i])
        modelos_tipo.append('Saco Vazio')


for i in range(len(volumes_tipo_)):# Modelos que aparecem após o Tipo_
    if volumes_tipo_[i][indice_qtde_6[i]+1] == 'Modelo':
        if volumes_tipo_[i][indice_qtde_6[i]+2] == '7':
            modelos_tipo_.append(indice_tipo_[i])
            modelos_tipo_.append('Mod 7') 
    if volumes_tipo_[i][indice_qtde_6[i]+1] == 'Branco':
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Sem Estampa') 
    if 'Cozinha' in volumes_tipo_[i]:
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Cozinha')
    if volumes_tipo_[i][indice_qtde_6[i]+1] == 'Burger':
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Burger') 
    if volumes_tipo_[i][indice_qtde_6[i]+1] == 'burger':
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Burger') 
    if 'Estampa' in volumes_tipo_[i]:
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Sem Estampa')
    if 'Como' in volumes_tipo_[i]:
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Mod 1')
    if 'Vazio' in volumes_tipo_[i]:
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Saco Vazio')
    if 'vazio' in volumes_tipo_[i]:
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Saco Vazio')
    if 'CBJ' in volumes_tipo_[i]:
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('CBJ')
    if volumes_tipo_[i][indice_qtde_6[i]+1] == 'branco':
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Sem Estampa')
    if volumes_tipo_[i][indice_qtde_6[i]+1] == 'cozinha':
        modelos_tipo_.append(indice_tipo_[i])
        modelos_tipo_.append('Cozinha')
    if 'Xadrez' in volumes_tipo_[i]:
        if 'Vermelho' in volumes_tipo_[i]:
            modelos_tipo_.append(indice_tipo_[i])
            modelos_tipo_.append('Xadrez Vermelho') 
        else:
            modelos_tipo_.append(indice_tipo_[i])
            modelos_tipo_.append('Xadrez Preto')
    if 'Losango' in volumes_tipo_[i]:
        if 'Vermelho' in volumes_tipo_[i]:
            modelos_tipo_.append(indice_tipo_[i])
            modelos_tipo_.append('Losango Vermelho')
        else:
            modelos_tipo_.append(indice_tipo_[i])
            modelos_tipo_.append('Losango Preto')


for i in range(len(volumes_cor)):# Modelos que aparecem após a Cor
    if 'Potato' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('The Potato Times')
    if 'Cozinha' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Cozinha')
    if 'Ecommerce' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Ecommerce')
    if 'estampa' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Sem Estampa')
    if 'Todos' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Batata para todos')
    if 'CBJ' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('CBJ')
    if 'Vazio' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Saco Vazio')
    if 'vazio' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Saco Vazio')
    if 'Estampa' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Sem Estampa')
    if 'Xadrez' in volumes_cor[i]:
        if 'Vermelho' in volumes_cor[i]:
            modelos_cor.append(indice_cor[i])
            modelos_cor.append('Xadrez Vermelho')
        else:
            modelos_cor.append(indice_cor[i])
            modelos_cor.append('Xadrez Preto')
    if 'Losango' in volumes_cor[i]:
        if 'Vermelho' in volumes_cor[i]:
            modelos_cor.append(indice_cor[i])
            modelos_cor.append('Losango Vermelho')
        else:
            modelos_cor.append(indice_cor[i])
            modelos_cor.append('Losango Preto')
    if 'Mãozinhas' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Mãozinhas')
    if 'Kit' in volumes_cor[i]:
        modelos_cor.append(indice_cor[i])
        modelos_cor.append('Vários')
    

for i in range(len(volumes_desenho)):# Modelos que aparecem após o desenho
    if 'CBJ' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('CBJ')
    if 'Vazio' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('Saco Vazio')
    if 'vazio' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('Saco Vazio')
    if volumes_desenho[i][indice_qtde_8[i]+1] == 'Modelo':
        if volumes_desenho[i][indice_qtde_8[i]+2] == '1':
            modelos_desenho.append(indice_desenho[i])
            modelos_desenho.append('Mod 1')
    if 'Como' in volumes_desenho[i][indice_qtde_8[i]+1]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('Mod 1')
    if 'Xadrez' in volumes_desenho[i][indice_qtde_8[i]+1]:#Modelo Xadrez
        if 'Vermelho' in volumes_desenho[i][indice_qtde_8[i]+2]:
            modelos_desenho.append(indice_desenho[i])
            modelos_desenho.append('Xadrez Vermelho')
        else:
            modelos_desenho.append(indice_desenho[i])
            modelos_desenho.append('Xadrez Preto')
    if 'Losango' in volumes_desenho[i][indice_qtde_8[i]+1]:#Modelo Losango
        if 'Vermelho' in volumes_desenho[i][indice_qtde_8[i]+2]:
            modelos_desenho.append(indice_desenho[i])
            modelos_desenho.append('Losango Vermelho')
        else:
            modelos_desenho.append(indice_desenho[i])
            modelos_desenho.append('Losango Preto')
    if 'Potato' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('The Potato Times')
    if 'Cozinha' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('Cozinha')
    if 'Ecommerce' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('Ecommerce')
    if 'estampa' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('Sem Estampa')
    if 'Kit' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('Vários')
    if 'Estampa' in volumes_desenho[i]:
        modelos_desenho.append(indice_desenho[i])
        modelos_desenho.append('Sem Estampa')
        
for i in range(len(volumes_cor1)):# Modelos que aparecem após a cor
    if 'Potato' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('The Potato Times')
    if 'Cozinha' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Cozinha')
    if 'Ecommerce' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Ecommerce')
    if 'estampa' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Sem Estampa')
    if 'Todos' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Batata para todos')
    if 'CBJ' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('CBJ')
    if 'Vazio' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Saco Vazio')
    if 'vazio' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Saco Vazio')
    if 'Estampa' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Sem Estampa')
    if 'Xadrez' in volumes_cor1[i]:
        if 'Vermelho' in volumes_cor1[i]:
            modelos_cor1.append(indice_cor[i])
            modelos_cor1.append('Xadrez Vermelho')
        else:
            modelos_cor1.append(indice_cor1[i])
            modelos_cor1.append('Xadrez Preto')
    if 'Losango' in volumes_cor1[i]:
        if 'Vermelho' in volumes_cor1[i]:
            modelos_cor1.append(indice_cor1[i])
            modelos_cor1.append('Losango Vermelho')
        else:
            modelos_cor1.append(indice_cor1[i])
            modelos_cor1.append('Losango Preto')
    if 'Mãozinhas' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Mãozinhas')
    if 'Kit' in volumes_cor1[i]:
        modelos_cor1.append(indice_cor1[i])
        modelos_cor1.append('Vários')
        
for i in range(len(volumes_estampa)):# Modelos que aparecem após a Estampa
    if volumes_estampa[i][indice_qtde_10[i]+1] == 'Sem':
        modelos_estampa.append(indice_estampa[i])
        modelos_estampa.append('Sem estampa')
        
for i in range(len(volumes_kraft)):# Modelos que aparecem após o Kraft
    if 'CBJ' in volumes_kraft[i]:
        modelos_kraft.append(indice_kraft[i])
        modelos_kraft.append('CBJ')
    if 'Vazio' in volumes_kraft[i]:
        modelos_kraft.append(indice_kraft[i])
        modelos_kraft.append('Saco Vazio')
    if 'Estampa' in volumes_kraft[i]:
        modelos_kraft.append(indice_kraft[i])
        modelos_kraft.append('Sem Estampa')


print(indice_tipo)
print(modelos_tipo)
print(indice_tipo_)
print(modelos_tipo_)
print(indice_cor)
print(modelos_cor)
print(indice_desenho)
print(modelos_desenho)
print(indice_cor1)
print(modelos_cor1)
print(indice_estampa)
print(modelos_estampa)
print(indice_kraft)
print(modelos_kraft)

[]
[]
[1, 2, 3, 4, 5, 6, 7, 10, 11, 18, 19, 21, 25, 26, 27]
[1, 'Xadrez Preto', 2, 'Sem Estampa', 3, 'Mod 1', 4, 'Sem Estampa', 5, 'CBJ', 6, 'CBJ', 7, 'Sem Estampa', 10, 'CBJ', 11, 'Burger', 18, 'Cozinha', 19, 'Sem Estampa', 21, 'Sem Estampa', 25, 'Xadrez Vermelho', 26, 'Xadrez Vermelho', 27, 'Sem Estampa']
[0, 8, 9, 12, 13, 15, 17, 20, 22, 24]
[0, 'Saco Vazio', 8, 'Saco Vazio', 9, 'CBJ', 12, 'Saco Vazio', 13, 'CBJ', 15, 'CBJ', 17, 'The Potato Times', 20, 'CBJ', 22, 'Sem Estampa', 24, 'Saco Vazio']
[0, 12, 13, 14, 15, 16, 20, 23, 24, 28]
[0, 'Saco Vazio', 12, 'Saco Vazio', 13, 'CBJ', 14, 'Xadrez Vermelho', 15, 'CBJ', 16, 'Mod 1', 20, 'CBJ', 23, 'Losango Vermelho', 24, 'Saco Vazio', 28, 'Mod 1']
[]
[]
[]
[]
[]
[]


In [24]:
# AGLOMERAMENTO FINAL



agrupamento_modelos = []#Junção dos modelos
a = 0
b = 0
c = 0
d = 0
e = 0
f = 0
g = 0

#Juntando as quantidades
for i in range(numero_volumes):#Pegando as quantidades na escrita das unidades com os índices
    if i in indice_tipo:
        agrupamento_modelos.append(i)
        agrupamento_modelos.append(modelos_tipo[c*2+1])
        c += 1

    if i in indice_tipo_:#Pegando as quantidades na escrita das caixas com os índices
        agrupamento_modelos.append(i)
        agrupamento_modelos.append(modelos_tipo_[a*2+1])
        a += 1
        
    if i in indice_cor:#Pegando as quantidades na escrita dos saches com os índices
        agrupamento_modelos.append(i)
        agrupamento_modelos.append(modelos_cor[b*2+1])
        b += 1
        
    if i in indice_desenho:
        agrupamento_modelos.append(i)
        agrupamento_modelos.append(modelos_desenho[d*2+1])
        d += 1
        
    if i in indice_cor1:#Pegando as quantidades na escrita da cor com os índices
        agrupamento_modelos.append(i)
        agrupamento_modelos.append(modelos_cor[e*2+1])
        e += 1
    
    if i in indice_estampa:
        agrupamento_modelos.append(i)
        agrupamento_modelos.append(modelos_estampa[f*2+1])
        f += 1
        
    if i in indice_kraft:
        agrupamento_modelos.append(i)
        agrupamento_modelos.append(modelos_kraft[g*2+1])
        g += 1

escrita_final = []# Escrita com todas informações

 # Retirando Volumes com mais de uma compra
    #Retirando 2 compras
for i in range(len(agrupamento_modelos)):
    if agrupamento_modelos.count(i) == 2:
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)
    #Retirando 3 compras
for i in range(len(agrupamento_modelos)):
    if agrupamento_modelos.count(i) == 3:
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)
    if agrupamento_modelos.count(i) == 4:
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)
        agrupamento_modelos.pop(i*2+2)

In [25]:
print(len(juntos))
print(len(agrupamento_modelos))
print(agrupamento_modelos)

29
58
[0, 'Saco Vazio', 1, 'Xadrez Preto', 2, 'Sem Estampa', 3, 'Mod 1', 4, 'Sem Estampa', 5, 'CBJ', 6, 'CBJ', 7, 'Sem Estampa', 8, 'Saco Vazio', 9, 'CBJ', 10, 'CBJ', 11, 'Burger', 12, 'Saco Vazio', 13, 'CBJ', 14, 'Xadrez Vermelho', 15, 'CBJ', 16, 'Mod 1', 17, 'The Potato Times', 18, 'Cozinha', 19, 'Sem Estampa', 20, 'CBJ', 21, 'Sem Estampa', 22, 'Sem Estampa', 23, 'Losango Vermelho', 24, 'Saco Vazio', 25, 'Xadrez Vermelho', 26, 'Xadrez Vermelho', 27, 'Sem Estampa', 28, 'Mod 1']


In [26]:
# AGLOMERAMENTO FINAL 2

agrupamento_modelos = [str(x) for x in agrupamento_modelos] #Passando para string
escrita_final = []

for i in range(numero_volumes):
    escrita_final.append(juntos[i]+ ' - ' + agrupamento_modelos[i*2+1])

print(escrita_final)

['4 Saco delivery - Saco Vazio', '2 Glassine - Xadrez Preto', '2 Guardanapo sachê - Sem Estampa', '6 kraft acoplado - Mod 1', '2 Guardanapo sachê - Sem Estampa', '1 Saco delivery - CBJ', '1 Saco delivery - CBJ', '3 Guardanapo sachê - Sem Estampa', '1 Saco delivery - Saco Vazio', '4 Saco P delivery - CBJ', '1 Saco delivery - CBJ', '1 Guardanapo sachê - Burger', '4 Saco delivery - Saco Vazio', '4 Saco delivery - CBJ', '2 Acoplado - Xadrez Vermelho', '4 Saco delivery - CBJ', '6 Acoplado - Mod 1', '1 Saquinho de batata kraft - The Potato Times', '2 Guardanapo sachê - Cozinha', '2 Guardanapo sachê - Sem Estampa', '4 Saco delivery - CBJ', '2 Guardanapo sachê - Sem Estampa', '0 Saco delivery - Sem Estampa', '2 Acoplado - Losango Vermelho', '4 Saco delivery - Saco Vazio', '1 Glassine - Xadrez Vermelho', '6 Acoplado - Xadrez Vermelho', '2 Guardanapo sachê - Sem Estampa', '2 Acoplado - Mod 1']


In [27]:
#Escrevendo arquivo

txt = open("testetxt.txt","w+")
for i in range(len(escrita_final)):
    txt.write(escrita_final[i])
    txt.write('\n')
    txt.write('\n')
txt.close()

In [28]:
escrita_final = [str(x) for x in escrita_final] #Passando para string
escrita_final1 = []#Escrita final 2.0
for i in range(numero_volumes):
    escrita_final1.append(str(volumes[i]) + ' ---> ' + escrita_final[i])

In [29]:
txt = open("teste1txt.txt","w+")
for i in range(len(escrita_final)):
    txt.write(escrita_final1[i])
    txt.write('\n')
    txt.write('\n')
txt.close()